In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 65kB/s 
     |████████████████████████████████| 204kB 19.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=8b0179f6ed1b44c3ad35b7138685d2af314f82a232b0c42535ba8594c760abbc
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [5]:
spark=SparkSession.builder.appName('dog_food').getOrCreate()

In [7]:
from google.colab import files
files.upload()

Saving dog_food.csv to dog_food.csv


{'dog_food.csv': b'A,B,C,D,Spoiled\n4,2,12.0,3,1.0\n5,6,12.0,7,1.0\n6,2,13.0,6,1.0\n4,2,12.0,1,1.0\n4,2,12.0,3,1.0\n10,3,13.0,9,1.0\n8,5,14.0,5,1.0\n5,8,12.0,8,1.0\n6,5,12.0,9,1.0\n3,3,12.0,1,1.0\n9,8,11.0,3,1.0\n1,10,12.0,3,1.0\n1,5,13.0,10,1.0\n2,10,12.0,6,1.0\n1,10,11.0,4,1.0\n5,3,12.0,2,1.0\n4,9,11.0,8,1.0\n5,1,11.0,1,1.0\n4,9,12.0,10,1.0\n5,8,10.0,9,1.0\n5,7,11.0,9,1.0\n4,10,13.0,8,1.0\n10,5,12.0,9,1.0\n2,4,13.0,4,1.0\n1,4,13.0,10,1.0\n1,8,12.0,1,1.0\n2,10,13.0,4,1.0\n6,2,12.0,4,1.0\n8,2,13.0,3,1.0\n6,4,12.0,2,1.0\n3,2,11.0,9,1.0\n10,6,12.0,10,1.0\n9,5,13.0,3,1.0\n9,2,12.0,5,1.0\n2,6,13.0,9,1.0\n4,2,12.0,10,1.0\n4,3,12.0,6,1.0\n7,1,12.0,1,1.0\n1,7,11.0,10,1.0\n9,2,11.0,10,1.0\n2,6,12.0,2,1.0\n9,4,11.0,5,1.0\n6,2,11.0,10,1.0\n3,10,11.0,4,1.0\n6,9,11.0,2,1.0\n10,6,11.0,9,1.0\n6,7,11.0,9,1.0\n7,2,13.0,8,1.0\n9,2,13.0,5,1.0\n8,7,12.0,6,1.0\n9,1,12.0,9,1.0\n3,5,14.0,3,1.0\n7,1,11.0,3,1.0\n5,9,12.0,7,1.0\n3,10,12.0,7,1.0\n9,8,13.0,9,1.0\n10,9,12.0,9,1.0\n10,7,11.0,2,1.0\n10,3,11.0,1,1.0

In [10]:
data=spark.read.csv('dog_food.csv',header='True',inferSchema='True')

In [11]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [12]:
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [17]:
assembler=VectorAssembler(inputCols=['A', 'B', 'C', 'D'],outputCol='features')

In [18]:
data2=assembler.transform(data)

In [19]:
data2.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
finaldata=data2.select('spoiled','features')
finaldata.printSchema()

root
 |-- spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [21]:
from pyspark.ml.classification import RandomForestClassifier

In [28]:
rfc=RandomForestClassifier(labelCol='spoiled',featuresCol='features',numTrees=100 )

In [29]:
rfc_model=rfc.fit(finaldata)

In [30]:
rfc_model.featureImportances  # which factor is affecting the most

SparseVector(4, {0: 0.0198, 1: 0.0188, 2: 0.9407, 3: 0.0207})

In [31]:
finaldata.head(1)

[Row(spoiled=1.0, features=DenseVector([4.0, 2.0, 12.0, 3.0]))]

In [32]:
data.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

In [ ]:
# result= chemical C is most affecting chemical